In [ ]:
import pickle as pkl
import numpy as np
from env import TransitNetworkEnv

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv

from agent import GNNPolicy, FeatureExtractor
from matplotlib import pyplot as plt

In [ ]:
env = TransitNetworkEnv()
model = PPO(
    tensorboard_log="./logs/",
    policy=GNNPolicy,
    env=env,
    verbose=1,
    policy_kwargs=dict(
        features_extractor_class=FeatureExtractor,
        features_extractor_kwargs=dict(hidden_dim=128, num_heads=4, out_dim=256, env=env),
    )
)

In [ ]:
model.learn(total_timesteps=100000, log_interval=10)